In [ ]:
!pip install fairseq

In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Baby.json.gz

In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz
!tar -xzvf roberta.large.mnli.tar.gz

In [5]:
### load the meta data
def parse_reviews_gzip(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

def parse_reviews_json(json_path):
  with open(json_path) as f:
    for line in f:
      yield eval(line)

reviews = parse_reviews_gzip("reviews_Baby_5.json.gz")
review = next(reviews)

In [7]:
from fairseq.models.roberta import RobertaModel

class SimularityCalculator:
  def __init__(self, dir='./roberta.large.mnli'):
    assert os.path.exists(dir + '/model.pt')
    roberta = RobertaModel.from_pretrained(dir, checkpoint_file='model.pt')
    roberta.eval()  # disable dropout (or leave in train mode to finetune)
    self.model = roberta
  
  def calculate_simularity(self, text, class_name):
    tokens = self.model.encode(text, f'this document is about {class_name.lower()}')
    logits = self.model.predict('mnli', tokens, return_logits=True)
    probabilities = logits.softmax(dim=-1).tolist()[0]
    entailment_probability = probabilities[2]
    return entailment_probability
    

In [ ]:
calculator = SimularityCalculator()

In [ ]:
print(calculator.calculate_simularity(review['reviewText'], 'Baby'))